# Running Tune experiments with BlendSearch and CFO

In this tutorial we introduce BlendSearch and CFO, while running a simple Ray Tune
experiment. Tune’s Search Algorithms integrate with FLAML and, as a result, allow
you to seamlessly scale up a BlendSearch and CFO optimization
process - without sacrificing performance.

Fast Library for Automated Machine Learning & Tuning (FLAML) does not rely on the
gradient of the objective function, but instead, learns from samples of the
search space. It is suitable for optimizing functions that are non-differentiable,
with many local minima, or even unknown but only testable. Therefore, it is
necessarily belongs to the domain of "derivative-free optimization"
and "black-box optimization".

FLAML has two primary algorithms: (1) Frugal Optimization for Cost-related
Hyperparameters (CFO) begins with a low-cost initial point and gradually moves to
a high-cost region as needed. It is a local search method that leverages randomized
direct search method with an adaptive step-size and random restarts.
As a local search method, it has an appealing provable convergence rate and bounded
cost but may get trapped in suboptimal local minima. (2) Economical Hyperparameter
Optimization With Blended Search Strategy (BlendSearch) combines CFO's local search
with global search, making it less suspectable to local minima traps.
It leverages the frugality of CFO and the space exploration ability of global search
methods such as Bayesian optimization.

In this example we minimize a simple objective to briefly demonstrate the usage of
FLAML with Ray Tune via `BlendSearch` and `CFO`. It's useful to keep in mind that
despite the emphasis on machine learning experiments, Ray Tune optimizes any implicit
or explicit objective. Here we assume `flaml==0.4.1` and `optuna==2.9.1` libraries
are installed. To learn more, please refer to
the [FLAML website](https://github.com/microsoft/FLAML/tree/main/flaml/tune).
  
Click below to see all the imports we need for this example.
You can also launch directly into a Binder instance to run this notebook yourself.
Just click on the rocket symbol at the top of the navigation.

In [1]:
import time

import ray
from ray import tune
from ray.air import session
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.flaml import BlendSearch, CFO

/Users/kai/.pyenv/versions/3.7.7/lib/python3.7/site-packages/flaml/searcher/blendsearch.py:14: DeprecationWarning: The module `ray.tune.suggest` has been moved to `ray.tune.search` and the old location will be deprecated soon. Please adjust your imports to point to the new location. Example: Do a global search and replace `ray.tune.suggest` with `ray.tune.search`.
  from ray.tune.suggest import Searcher
/Users/kai/.pyenv/versions/3.7.7/lib/python3.7/site-packages/flaml/searcher/blendsearch.py:15: DeprecationWarning: The module `ray.tune.suggest.optuna` has been moved to `ray.tune.search.optuna` and the old location will be deprecated soon. Please adjust your imports to point to the new location. Example: Do a global search and replace `ray.tune.suggest.optuna` with `ray.tune.search.optuna`.
  from ray.tune.suggest.optuna import OptunaSearch as GlobalSearch
/Users/kai/.pyenv/versions/3.7.7/lib/python3.7/site-packages/flaml/tune/sample.py:39: DeprecationWarning: The module `ray.tune.samp

Let's start by defining a simple evaluation function.
We artificially sleep for a bit (`0.1` seconds) to simulate a long-running ML experiment.
This setup assumes that we're running multiple `step`s of an experiment and try to
tune three hyperparameters, namely `width` and `height`, and `activation`.

In [2]:
def evaluate(step, width, height, activation):
    time.sleep(0.1)
    activation_boost = 10 if activation=="relu" else 1
    return (0.1 + width * step / 100) ** (-1) + height * 0.1 + activation_boost

Next, our `objective` function takes a Tune `config`, evaluates the `score` of your
experiment in a training loop, and uses `session.report` to report the `score` back to Tune.

In [3]:
def objective(config):
    for step in range(config["steps"]):
        score = evaluate(step, config["width"], config["height"], config["activation"])
        session.report({"iterations": step, "mean_loss": score})

In [4]:
ray.init(configure_logging=False)

Python version:,3.7.7
Ray version:,3.0.0.dev0
Dashboard:,http://127.0.0.1:8265


## Running Tune experiments with BlendSearch

This example demonstrates the usage of Economical Hyperparameter Optimization
With Blended Search Strategy (BlendSearch) with Ray Tune.

Now we define the search algorithm built from `BlendSearch`, constrained to a
maximum of `4` concurrent trials with a `ConcurrencyLimiter`.

In [5]:
algo = BlendSearch()
algo = ConcurrencyLimiter(algo, max_concurrent=4)

The number of samples this Tune run is set to `1000`.
(you can decrease this if it takes too long on your machine).

In [6]:
num_samples = 1000

In [7]:
# If 1000 samples take too long, you can reduce this number.
# We override this number here for our smoke tests.
num_samples = 10

Next we define a search space. The critical assumption is that the optimal
hyperparameters live within this space. Yet, if the space is very large, then those
hyperparameters may be difficult to find in a short amount of time.

In [8]:
search_config = {
    "steps": 100,
    "width": tune.uniform(0, 20),
    "height": tune.uniform(-100, 100),
    "activation": tune.choice(["relu, tanh"])
}

Finally, we run the experiment to `"min"`imize the "mean_loss" of the `objective` by
searching `search_config` via `algo`, `num_samples` times. This previous sentence is
fully characterizes the search problem we aim to solve. With this in mind, observe
how efficient it is to execute `tuner.fit()`.

In [9]:
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        num_samples=num_samples,
    ),
    param_space=search_config,
)
results = tuner.fit()

Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
[I 2022-07-22 15:18:10,659] A new study created in memory with name: optuna


Trial name,status,loc,activation,height,steps,width,loss,iter,total time (s),iterations,neg_mean_loss
objective_1e0f814e,TERMINATED,127.0.0.1:45801,"relu, tanh",29.4532,100,1.94864,4.43814,100,11.061,99,-4.43814
objective_1fc231f8,TERMINATED,127.0.0.1:45809,"relu, tanh",-95.8496,100,15.4264,-8.51991,100,11.6807,99,8.51991
objective_1fc3b668,TERMINATED,127.0.0.1:45810,"relu, tanh",49.7608,100,12.673,6.05515,100,11.7372,99,-6.05515
objective_1fc58b1e,TERMINATED,127.0.0.1:45811,"relu, tanh",-55.0407,100,9.97014,-4.40377,100,11.6335,99,4.40377
objective_265c09ee,TERMINATED,127.0.0.1:45837,"relu, tanh",52.1061,100,3.96126,6.45927,100,10.7539,99,-6.45927
objective_28587494,TERMINATED,127.0.0.1:45842,"relu, tanh",-82.332,100,3.38222,-6.94321,100,10.7116,99,6.94321
objective_28682bfa,TERMINATED,127.0.0.1:45845,"relu, tanh",-94.9771,100,17.681,-8.4409,100,10.7471,99,8.4409
objective_28788388,TERMINATED,127.0.0.1:45848,"relu, tanh",90.6787,100,13.7072,10.141,100,10.7525,99,-10.141
objective_2e3cd63e,TERMINATED,127.0.0.1:45864,"relu, tanh",2.43845,100,0.0789653,6.85628,100,10.7006,99,-6.85628
objective_3046455a,TERMINATED,127.0.0.1:45869,"relu, tanh",22.5052,100,16.2524,3.31229,100,10.7176,99,-3.31229


Result for objective_1e0f814e:
  date: 2022-07-22_15-18-14
  done: false
  experiment_id: 623da6c1aa28400a9d85da19bdc5721d
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 13.945321626643842
  neg_mean_loss: -13.945321626643842
  node_ip: 127.0.0.1
  pid: 45801
  time_since_restore: 0.10402607917785645
  time_this_iter_s: 0.10402607917785645
  time_total_s: 0.10402607917785645
  timestamp: 1658499494
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 1e0f814e
  warmup_time: 0.003920078277587891
  
Result for objective_1fc58b1e:
  date: 2022-07-22_15-18-17
  done: false
  experiment_id: da508e209a1946e89ec15a3f1a5ce5ef
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 5.495932910616953
  neg_mean_loss: -5.495932910616953
  node_ip: 127.0.0.1
  pid: 45811
  time_since_restore: 0.10269379615783691
  time_this_iter_s: 0.10269379615783691
  time_total_s: 0.10269379615783691
  timestamp: 16

Result for objective_1fc3b668:
  date: 2022-07-22_15-18-29
  done: true
  experiment_id: 5cdd489afb0e43b7a44aa4484765dc92
  experiment_tag: 3_activation=relu_tanh,height=49.7608,steps=100,width=12.6730
  hostname: Kais-MacBook-Pro.local
  iterations: 99
  iterations_since_restore: 100
  mean_loss: 6.055152568795227
  neg_mean_loss: -6.055152568795227
  node_ip: 127.0.0.1
  pid: 45810
  time_since_restore: 11.73720097541809
  time_this_iter_s: 0.10816287994384766
  time_total_s: 11.73720097541809
  timestamp: 1658499509
  timesteps_since_restore: 0
  training_iteration: 100
  trial_id: 1fc3b668
  warmup_time: 0.002666950225830078
  
Result for objective_28587494:
  date: 2022-07-22_15-18-31
  done: false
  experiment_id: 750bd707fda147f0bd615c32243d64e5
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 2.766796283480206
  neg_mean_loss: -2.766796283480206
  node_ip: 127.0.0.1
  pid: 45842
  time_since_restore: 0.10265803337097168
  time_this_i

Result for objective_28788388:
  date: 2022-07-22_15-18-42
  done: true
  experiment_id: e959dae493a240c489a3193352c46f3a
  experiment_tag: 8_activation=relu_tanh,height=90.6787,steps=100,width=13.7072
  hostname: Kais-MacBook-Pro.local
  iterations: 99
  iterations_since_restore: 100
  mean_loss: 10.141019147716873
  neg_mean_loss: -10.141019147716873
  node_ip: 127.0.0.1
  pid: 45848
  time_since_restore: 10.752525091171265
  time_this_iter_s: 0.10643196105957031
  time_total_s: 10.752525091171265
  timestamp: 1658499522
  timesteps_since_restore: 0
  training_iteration: 100
  trial_id: '28788388'
  warmup_time: 0.0027070045471191406
  
Result for objective_3046455a:
  date: 2022-07-22_15-18-44
  done: false
  experiment_id: f282080699d848f69c4b8626bd4c2d91
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 13.250521336587763
  neg_mean_loss: -13.250521336587763
  node_ip: 127.0.0.1
  pid: 45869
  time_since_restore: 0.10329103469848633
  ti

Here are the hyperparamters found to minimize the mean loss of the defined objective.

In [10]:
print("Best hyperparameters found were: ", results.get_best_result().config)

Best hyperparameters found were:  {'steps': 100, 'width': 15.42641286533492, 'height': -95.8496101281197, 'activation': 'relu, tanh'}


## Incorporating a time budget to the experiment

Define the time budget in seconds:

In [11]:
time_budget_s = 30

Similarly we define a search space, but this time we feed it as an argument to
`BlendSearch` rather than `Tuner()`'s `param_space` argument.

We next define the time budget via `set_search_properties`.
And once again include the `ConcurrencyLimiter`.

In [12]:
algo = BlendSearch(
    metric="mean_loss",
    mode="min",
    space={
        "width": tune.uniform(0, 20),
        "height": tune.uniform(-100, 100),
        "activation": tune.choice(["relu", "tanh"]),
    },
)
algo.set_search_properties(config={"time_budget_s": time_budget_s})
algo = ConcurrencyLimiter(algo, max_concurrent=4)

You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `param_space` argument to `Tuner()` instead.
[I 2022-07-22 15:18:55,531] A new study created in memory with name: optuna


Now we run the experiment, this time with the `time_budget` included as an argument.
Note: We allow for virtually infinite `num_samples` by passing `-1`, so that the
experiment is stopped according to the time budget rather than a sample limit.

In [13]:
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        num_samples=-1,
        time_budget_s=time_budget_s,
    ),
    param_space={"steps": 100},
)
results = tuner.fit()

Trial name,status,loc,activation,height,steps,width,loss,iter,total time (s),iterations,neg_mean_loss
objective_38449df6,TERMINATED,127.0.0.1:45883,relu,29.4532,100,1.94864,13.4381,100,10.6555,99,-13.4381
objective_39d44fea,TERMINATED,127.0.0.1:45889,tanh,-95.8496,100,15.4264,-8.51991,100,11.2945,99,8.51991
objective_39d5c046,TERMINATED,127.0.0.1:45890,tanh,-55.0407,100,9.97014,-4.40377,100,11.169,99,4.40377
objective_39d71608,TERMINATED,127.0.0.1:45891,tanh,-82.332,100,3.38222,-6.94321,100,11.3021,99,6.94321
objective_402f9534,TERMINATED,127.0.0.1:45909,relu,2.43845,100,0.0789653,15.8563,100,10.7275,99,-15.8563
objective_4208c6aa,TERMINATED,127.0.0.1:45914,relu,-41.6248,100,14.4351,5.90701,100,10.811,99,-5.90701
objective_421aeb64,TERMINATED,127.0.0.1:45917,relu,-94.9771,100,17.681,0.559098,100,10.8502,99,-0.559098
objective_421d81ee,TERMINATED,127.0.0.1:45918,tanh,-96.7222,100,13.1718,-8.59611,100,10.8456,99,8.59611
objective_481b0de6,TERMINATED,127.0.0.1:45933,relu,-81.3401,100,15.2514,2.48132,11,1.17882,10,-2.48132
objective_4a00fc10,TERMINATED,127.0.0.1:45940,tanh,-100,100,11.0922,,,,,


Result for objective_38449df6:
  date: 2022-07-22_15-18-58
  done: false
  experiment_id: 22ed7d2e0f154e3d89fddc8f7f0b743f
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 22.945321626643842
  neg_mean_loss: -22.945321626643842
  node_ip: 127.0.0.1
  pid: 45883
  time_since_restore: 0.1004023551940918
  time_this_iter_s: 0.1004023551940918
  time_total_s: 0.1004023551940918
  timestamp: 1658499538
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 38449df6
  warmup_time: 0.0025548934936523438
  
Result for objective_39d5c046:
  date: 2022-07-22_15-19-00
  done: false
  experiment_id: faa00d9b1a2b4014bf01bab455bf9dbc
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 5.495932910616953
  neg_mean_loss: -5.495932910616953
  node_ip: 127.0.0.1
  pid: 45890
  time_since_restore: 0.10018515586853027
  time_this_iter_s: 0.10018515586853027
  time_total_s: 0.10018515586853027
  timestamp: 1658

Result for objective_4208c6aa:
  date: 2022-07-22_15-19-14
  done: false
  experiment_id: ce292c59f7c74e1ab42de6004eb3d846
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 15.837521363412662
  neg_mean_loss: -15.837521363412662
  node_ip: 127.0.0.1
  pid: 45914
  time_since_restore: 0.10442900657653809
  time_this_iter_s: 0.10442900657653809
  time_total_s: 0.10442900657653809
  timestamp: 1658499554
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 4208c6aa
  warmup_time: 0.002707958221435547
  
Result for objective_421aeb64:
  date: 2022-07-22_15-19-14
  done: false
  experiment_id: a4b264c05660423ea645cef7a9cd38bb
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 10.502293516994555
  neg_mean_loss: -10.502293516994555
  node_ip: 127.0.0.1
  pid: 45917
  time_since_restore: 0.1030268669128418
  time_this_iter_s: 0.1030268669128418
  time_total_s: 0.1030268669128418
  timestamp: 165

In [14]:
print("Best hyperparameters found were: ", results.get_best_result().config)

Best hyperparameters found were:  {'steps': 100, 'width': 13.171830039895717, 'height': -96.72215542618497, 'activation': 'tanh'}


## Running Tune experiments with CFO

This example demonstrates the usage of Frugal Optimization for Cost-related
Hyperparameters (CFO) with Ray Tune.

We now define the search algorithm as built from `CFO`, constrained to a maximum of `4`
concurrent trials with a `ConcurrencyLimiter`.

In [15]:
algo = CFO()
algo = ConcurrencyLimiter(algo, max_concurrent=4)

The number of samples is the number of hyperparameter combinations that will be
tried out. This Tune run is set to `1000` samples.
(you can decrease this if it takes too long on your machine).

In [16]:
num_samples = 1000

In [17]:
# If 1000 samples take too long, you can reduce this number.
# We override this number here for our smoke tests.
num_samples = 10

Next we define a search space. The critical assumption is that the optimal
hyperparameters live within this space. Yet, if the space is very large, then
those hyperparameters may be difficult to find in a short amount of time.

In [18]:
search_config = {
    "steps": 100,
    "width": tune.uniform(0, 20),
    "height": tune.uniform(-100, 100),
    "activation": tune.choice(["relu, tanh"])
}

Finally, we run the experiment to `"min"`imize the "mean_loss" of the `objective`
by searching `search_config` via `algo`, `num_samples` times. This previous sentence
is fully characterizes the search problem we aim to solve. With this in mind,
notice how efficient it is to execute `tuner.fit()`.

In [19]:
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        num_samples=num_samples,
    ),
    param_space=search_config,
)
results = tuner.fit()

Trial name,status,loc,activation,height,steps,width,loss,iter,total time (s),iterations,neg_mean_loss
objective_4bc281fe,TERMINATED,127.0.0.1:45958,"relu, tanh",29.4532,100,1.94864,4.43814,100,10.7193,99,-4.43814
objective_4d4ed536,TERMINATED,127.0.0.1:45963,"relu, tanh",-26.2681,100,7.25618,-1.48952,100,11.1932,99,1.48952
objective_4d5064aa,TERMINATED,127.0.0.1:45964,"relu, tanh",-46.7215,100,15.1097,-3.60574,100,11.1983,99,3.60574
objective_4d51b38c,TERMINATED,127.0.0.1:45965,"relu, tanh",-97.2777,100,15.1242,-8.66142,100,11.1977,99,8.66142
objective_53b39e84,TERMINATED,127.0.0.1:45983,"relu, tanh",25.7812,100,1.77704,4.11597,100,10.7964,99,-4.11597
objective_558c6132,TERMINATED,127.0.0.1:45990,"relu, tanh",33.1252,100,2.12024,4.76727,100,10.7659,99,-4.76727
objective_558f27aa,TERMINATED,127.0.0.1:45991,"relu, tanh",40.929,100,0.0138032,13.8907,100,10.785,99,-13.8907
objective_5592ca68,TERMINATED,127.0.0.1:45992,"relu, tanh",17.9775,100,3.88348,3.05125,100,10.7802,99,-3.05125
objective_5b999414,TERMINATED,127.0.0.1:46011,"relu, tanh",11.0354,100,3.27423,2.40281,100,12.6262,99,-2.40281
objective_5d8462c2,TERMINATED,127.0.0.1:46018,"relu, tanh",24.9195,100,4.49273,3.71184,100,10.7339,99,-3.71184


Result for objective_4bc281fe:
  date: 2022-07-22_15-19-30
  done: false
  experiment_id: 129b24a058ee40fd991fb40844988057
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 13.945321626643842
  neg_mean_loss: -13.945321626643842
  node_ip: 127.0.0.1
  pid: 45958
  time_since_restore: 0.10440444946289062
  time_this_iter_s: 0.10440444946289062
  time_total_s: 0.10440444946289062
  timestamp: 1658499570
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 4bc281fe
  warmup_time: 0.003987789154052734
  
Result for objective_4d4ed536:
  date: 2022-07-22_15-19-33
  done: false
  experiment_id: 6c18c359d1574792ac99ed37265d5d87
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 8.373189005362395
  neg_mean_loss: -8.373189005362395
  node_ip: 127.0.0.1
  pid: 45963
  time_since_restore: 0.10463309288024902
  time_this_iter_s: 0.10463309288024902
  time_total_s: 0.10463309288024902
  timestamp: 16

Result for objective_558c6132:
  date: 2022-07-22_15-19-47
  done: false
  experiment_id: 98df2ca60c5f4a33a3da6870c3d8db4c
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 14.31252243200564
  neg_mean_loss: -14.31252243200564
  node_ip: 127.0.0.1
  pid: 45990
  time_since_restore: 0.10317206382751465
  time_this_iter_s: 0.10317206382751465
  time_total_s: 0.10317206382751465
  timestamp: 1658499587
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 558c6132
  warmup_time: 0.003049135208129883
  
Result for objective_558f27aa:
  date: 2022-07-22_15-19-47
  done: false
  experiment_id: 73def1ffc7f443669328b88695453248
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 15.092898207127158
  neg_mean_loss: -15.092898207127158
  node_ip: 127.0.0.1
  pid: 45991
  time_since_restore: 0.10495114326477051
  time_this_iter_s: 0.10495114326477051
  time_total_s: 0.10495114326477051
  timestamp: 16

Result for objective_5d8462c2:
  date: 2022-07-22_15-20-00
  done: false
  experiment_id: 37975742a6894daeb580d222effadf3c
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 13.491949373944276
  neg_mean_loss: -13.491949373944276
  node_ip: 127.0.0.1
  pid: 46018
  time_since_restore: 0.1021728515625
  time_this_iter_s: 0.1021728515625
  time_total_s: 0.1021728515625
  timestamp: 1658499600
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 5d8462c2
  warmup_time: 0.0030527114868164062
  
Result for objective_5b999414:
  date: 2022-07-22_15-20-02
  done: false
  experiment_id: 5548713c4f5046c0a44e1946db840d73
  hostname: Kais-MacBook-Pro.local
  iterations: 29
  iterations_since_restore: 30
  mean_loss: 3.0563505542773735
  neg_mean_loss: -3.0563505542773735
  node_ip: 127.0.0.1
  pid: 46011
  time_since_restore: 5.116016149520874
  time_this_iter_s: 0.1060340404510498
  time_total_s: 5.116016149520874
  timestamp: 1658499602
  t

Here are the hyperparameters found to minimize the mean loss of the defined objective.

In [20]:
print("Best hyperparameters found were: ", results.get_best_result().config)

Best hyperparameters found were:  {'steps': 100, 'width': 15.124213652112319, 'height': -97.27768667042203, 'activation': 'relu, tanh'}


In [21]:
ray.shutdown()